<img style="float: left" src="images/spark.png" />
<img style="float: right" src="images/surfsara.png" />
<hr style="clear: both" />

# Introduction to Apache Spark

In this first notebook we will explore Spark using the Python API, called PySpark.

_You can edit the cells below and execute the code by selecting the cell and press Shift-Enter. Code completion is supported by use of the Tab key._

During the exercises you may want to refer to the [PySpark documentation](https://spark.apache.org/docs/1.6.1/api/python/pyspark.html#pyspark.RDD) for more information on possible transformations and actions.

In [ ]:
# initialize Spark
from pyspark import SparkContext, SparkConf

if not 'sc' in globals(): # This Python 'trick' makes sure the SparkContext sc is initialized exactly once
    conf = SparkConf().setMaster('local[*]')
    sc = SparkContext(conf=conf)

## Creating an RDD from a list

All parallel work in Spark is done on RDDs, so the first thing we need to do is convert our data (in this case a list) to an RDD. We will use the `parallelize` method on the `SparkContext sc`. It takes two arguments: (1) the collection, and (2) the number of partitions (splits). The second argument is optional.

In [ ]:
animal_list = ['dog', 'cat', 'rabbit', 'hare', 'deer', 'gull', 'woodpecker', 'mole']
animal_rdd = sc.parallelize(animal_list, 2)

print('the type of words_list is: ' + str(type(animal_list)))
print('the type of words_rdd is: ' + str(type(animal_rdd)))

## Map transformation

We now want to change all the words in the `animal_rdd` to their plural form. We will do this using a [map](https://spark.apache.org/docs/1.6.1/api/python/pyspark.html?highlight=map#pyspark.RDD.map) transformation.
Remember that the map function will apply the function to each element of the RDD.

First, you should write a simple function that takes a single word as argument and return the word with an 's' added to it. In the next step we will use this function in a map transformation of the `animal_rdd`.

In [ ]:
def make_plural(word):
    """Adds an 's' to `word`.

    Note:
        This is a simple function that only adds an 's'.  

    Args:
        word (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
### BEGIN SOLUTION
    return word + 's'
### END SOLUTION

print(make_plural('cat'))

Next, you should use the `make_plural` function as input for the map transformation on `animal_rdd`.

The action [`collect`](https://spark.apache.org/docs/1.6.1/api/python/pyspark.html?highlight=collect#pyspark.RDD.collect) transfers the content of the RDD to the driver. Note, that a large RDD may be scattered over many machines. In such a case a `collect` can be a very bad idea.

In [ ]:
# Your solution should be of the form
# plural_rdd = animal_rdd.map(<FILL IN>)

### BEGIN SOLUTION
plural_rdd = animal_rdd.map(make_plural)
### END SOLUTION

print(plural_rdd.collect())

## Using lambda functions

But wait a minute! We can actually achieve the same functionality by using [lambda functions](http://www.diveintopython.net/power_of_introspection/lambda_functions.html). In this case we rewrite the body of `make_plural` into a lambda function.

Hint: The map function needs a function as argument. This function needs one argument, let's call that `x`. The body of the function adds an 's' to the end of `x`.

In [ ]:
# A lambda function for adding s at the end of a string
# lambda_plural_rdd = animal_rdd.map(<FILL IN>)

### BEGIN SOLUTION
lambda_plural_rdd = animal_rdd.map(lambda x: x + 's')
### END SOLUTION

print(lambda_plural_rdd.collect())

Let's do another map transformation. For each word in `animal` determine its length. The Python function `len`  will return the length of a string.

You can do this with a lambda function, but there is another way.

In [ ]:
### BEGIN SOLUTION
word_lengths = animal_rdd.map(len)
### END SOLUTION

print(word_lengths.collect())

Test your solution by running the following cell:

In [ ]:
from test_helper import Test

Test.assertEquals(sorted(word_lengths.collect()), [3, 3, 4, 4, 4, 4, 6, 10], 'incorrect value for word_lengths')

 ## RDD from a text file

Manipulating a list with eight elements gets boring pretty fast, so let's move start processing a file. In this part we will read the file `alice.txt` and will count the number of occurrences of every word. This 'Word Count' is the ['Hello World'](https://en.wikipedia.org/wiki/%22Hello,_World!%22_program) of data-processing frameworks.

In [ ]:
# create an RDD from alice.txt where every element is a line of the file
alice_rdd = sc.textFile('file://///home/jovyan/work/data/alice.txt').cache()

## `collect` call not accepted

As we mentioned before, once your data no longer fits on the screen the `collect` method becomes less useful or even problematic. But we still want to have a way to inspect the (intermediate) results. For this we can use one of the following methods as a replacement of `collect`:

- `first()`: return the first elements of the RDD
- `take(n)`:  return a list of `n` elements
- `takeOrdered(n, [key=f])`: return the first `n` elements of the RDD, the order is defined by the optional function `f`.

In [ ]:
# from now on we will use the pprint module for more readable output
import pprint
pp = pprint.PrettyPrinter(indent=2)

print('first: ' + alice_rdd.first())
print('take(5): ')
pp.pprint(alice_rdd.take(5))

You can ignore the 'u' in front of all the elements. This is Python specifying the encoding (Unicode) of the strings.

In [ ]:
print(alice_rdd.take(5))

## The one-to-many map: flatMap

We have an RDD of lines, so let's try to convert this to an RDD of words by splitting the lines on whitespace:

In [ ]:
alice_words_try = alice_rdd.map(lambda line: line.split())

pp.pprint(alice_words_try.take(4))

This doesn't look right! We want an RDD of words, but we have created an RDD of lists of words: one extra level of nesting.

We want to map a function on an input that returns multiple values in a list, but then not to want the output nested in the same way as the input was. As it is commonly needed Spark includes a [`flatMap`](https://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=map#pyspark.RDD.flatMap) transformation that will _flatten_ the output of the function.

In [ ]:
alice_words = alice_rdd.flatMap(lambda line: line.split())

pp.pprint(alice_words.take(4))

## Key-Value Pairs

In order to count words in parallel we are going to use an RDD which consist of simple key-value pairs. We will call this RDD `alice_pairs` and it will be result of a transformation of `alice_words`. For every word in `alice_words` we want to have a `(word, 1)` tuple.

In [ ]:
### BEGIN SOLUTION
alice_pairs = alice_words.map(lambda word: (word, 1))
### END SOLUTION

print(alice_pairs.take(10))

In [ ]:
Test.assertEquals(alice_pairs.take(10),
                  [(u'The', 1), (u'Project', 1), (u'Gutenberg', 1), (u'EBook', 1), (u'of', 1),
                   (u'Alice', 1), (u'in', 1), (u'Wonderland,', 1), (u'by', 1), (u'Lewis', 1)])

## ReduceByKey

Next, we are going to count all words by using [`reduceByKey`](https://spark.apache.org/docs/1.6.0/api/python/pyspark.html?highlight=reducebykey#pyspark.RDD.reduceByKey).

`ReducebyKey` expects the RDD to consist of key-value pairs an it will perform a reduce operation per key.
It will need a two-argument function as input that will work on the values only. Remember that a reduce function needs two arguments and will reduce all elements to a single value.

In [ ]:
# Note that reduceByKey takes in a function that accepts two values and returns a single value
# The function that is input to reduceByKey only works on the values. Spark will execute this function per key

### BEGIN SOLUTION
word_counts = alice_pairs.reduceByKey(lambda x, y: x + y)
### END SOLUTION

top10_words = word_counts.takeOrdered(10, key=lambda p: -p[1])

print(top10_words)

In [ ]:
# TEST Counting using reduceByKey
Test.assertEquals(top10_words,
                  [(u'the', 577), (u'and', 296), (u'a', 265), (u'to', 241), (u'she', 197),
                   (u'of', 197), (u'was', 159), (u'in', 157), (u'said', 129), (u'it', 114)],
                  'incorrect value for top10_words')

## ReduceByKey - background

The `reduceByKey` method is similar to Hadoop's Reduce, but more restrictive. The function you provide to `reduceByKey` needs to be both [commutative](https://en.wikipedia.org/wiki/Commutative_property) and [associative](https://en.wikipedia.org/wiki/Associative_Property).

These restrictions allow Spark to perform additional optimisations, performing the operation on each partitions of the RDD and minimising network traffic. In the example below at most six values need to be transmitted.

![reduceByKey](images/reducebykey.png)
(Picture by DataBricks)

## End of the first notebook

Congratulations! You finished the first notebook. You can continue to the next notebook "02 - Counting Kmers with Spark". Close this notebook via 'File' -> 'Close and Halt' to stop the underlying kernel and release computing resources.